In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import pymatgen as pymat
%matplotlib inline
#from biokit.viz import corrplot
from sklearn import linear_model, ensemble, gaussian_process
import seaborn as sns
from sklearn import metrics, decomposition, datasets, preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
#from sklearn.grid_search import GridSearchCV
from scipy.interpolate import UnivariateSpline
from mpl_toolkits.mplot3d import Axes3D
from sklearn.utils import shuffle, resample
from scipy.stats.kde import gaussian_kde
from sklearn.kernel_ridge import KernelRidge
from numpy import loadtxt
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor
import warnings; warnings.simplefilter('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

plt.rcParams['font.size'] = 24
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = (12,12)
plt.rcParams['axes.linewidth'] = 6.0


#File Imports
filePath = ''  # (° ͜ʖ ͡°) 
df_train = pd.read_csv(filePath + 'data/equip_failures_training_set.csv')
df_test = pd.read_csv(filePath + 'data/equip_failures_test_set.csv')

In [12]:
df_train.replace('na', 'nan', inplace = True)
df_test.replace('na', 'nan', inplace = True)
filter_col = [col for col in df_train if col.startswith('sensor')]
descriptors = df_train[filter_col].astype('float32')
target=df_train["target"].astype('float32')

In [31]:
import numpy as np
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(descriptors)
imp_descriptors=pd.DataFrame(imp_mean.transform(descriptors))
imp_descriptors.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169
0,76698.0,0.713189,2.130706e+09,280.000000,0.0,0.0,0.0,0.0,0.0,0.0,37250.0,1432864.0,3664156.0,1007684.0,25896.0,0.0,2551696.0,0.0,0.0,0.0,0.0,0.0,4933296.0,3655166.0,1766008.0,1132040.0,0.0,0.0,0.0,0.0,1012.0,268.0,0.0,0.0,0.0,0.0,0.0,469014.0,4239660.0,703300.0,755876.0,0.0,5374.0,2108.0,4114.0,12348.0,615248.0,5526276.0,2378.0,4.0,0.0,0.0,2328746.0,1022304.0,415432.0,287230.0,310246.0,681504.0,1118814.0,3574.0,0.0,0.0,6700214.0,0.0,10.0,108.0,50.0,2551696.0,97518.0,947550.0,799478.0,330760.000000,353400.000000,299160.000000,305200.000000,283680.000000,551389.799347,582871.32293,604886.613264,178540.0,76698.078125,6700214.0,6700214.0,6599892.0,43566.0,68656.0,54064.0,638360.0,6167850.0,1209600.0,246244.0,2.000000,96.000000,0.000000,5.245752e+06,0.0,916567.687500,6.0,1924.0,0.0,0.0,0.0,118196.0,1309472.0,3247182.0,1381362.0,98822.0,11208.0,1608.0,220.000000,240.0,6700214.0,37.06298,10476.0,1226.0,267998.0,521832.0,428776.0,4015854.0,895240.0,26330.0,118.0,0.0,532.000000,734.000000,4.122704e+06,51288.000000,0.000000,532572.000000,0.000000,18.000000,5.330690e+06,4732.0,1126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62282.0,85908.0,32790.0,0.0,0.0,202710.0,37928.0,14745580.0,1876644.0,0.0,0.0,0.0,0.0,2801180.0,2445.800049,2712.0,965866.0,1706908.0,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,33058.0,0.713189,0.000000e+00,190620.644675,0.0,0.0,0.0,0.0,0.0,0.0,18254.0,653294.0,1720800.0,516724.0,31642.0,0.0,1393352.0,0.0,68.0,0.0,0.0,0.0,2560898.0,2127150.0,1084598.0,338544.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71510.0,772720.0,1996924.0,99560.0,0.0,7336.0,7808.0,13776.0,13086.0,1010074.0,1873902.0,14726.0,6.0,0.0,0.0,1378576.0,447166.0,199512.0,154298.0,137280.0,138668.0,165908.0,229652.0,87082.0,4708.0,3646660.0,86.0,454.0,364.0,350.0,1393352.0,49028.0,688314.0,392208.0,341420.000000,359780.000000,366560.000000,463710.833594,513147.820297,551389.799347,582871.32293,604886.613264,6700.0,33057.511719,3646660.0,3646660.0,3582034.0,17733.0,260120.0,115626.0,6900.0,2942850.0,1209600.0,0.0,190221.816478,91.520415,0.000443,2.291079e+06,0.0,643536.937500,0.0,0.0,0.0,0.0,38.0,98644.0,1179502.0,1286736.0,336388.0,36294.0,5192.0,56.0,190515.572299,0.0,3646660.0,37.06298,6160.0,796.0,164860.0,350066.0,272956.0,1837600.0,301242.0,9148.0,22.0,0.0,749.091271,1222.960989,1.928825e+06,351510.236275,274.179338,19374.290483,7.393791,13.416652,2.200752e+06,3312.0,522.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33736.0,36946.0,5936.0,0.0,0.0,103330.0,16254.0,4510080.0,868538.0,0.0,0.0,0.0,0.0,3477820.0,2211.760010,2334.0,664504.0,824154.0,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,41040.0,0.713189,2.280000e+02,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,1648.0,370592.0,1883374.0,292936.0,12016.0,0.0,1234132.0,0.0,0.0,0.0,0.0,0.0,2371990.0,2173634.0,300796.0,153698.0,0.0,0.0,0.0,0.0,358.0,110.0,0.0,0.0,0.0,0.0,0.0,0.0,870456.0,239798.0,1450312.0,0.0,1620.0,1156.0,1228.0,34250.0,1811606.0,710672.0,34.0,0.0,0.0,0.0,790690.0,672026.0,332340.0,254892.0,189596.0,135758.0,103552.0,81666.0,46.0,0.0,2673338.0,128.0,202.0,576.0,4.0,1234132.0,28804.0,160176.0,139730.0,137160.000000,130640.000000,399603.167571,463710.833594,513147.820297,551389.799347,582871.32293,604886.613264,28000.0,41040.078125,2673338.0,2673338.0,2678534.0,15439.0,7466.0,22436.0,248240.0,2560566.0,1209600.0,63328.0,0.000000,124.000000,0.000000,2.322692e+06,0.0,236099.515625,0.0,0.0,

In [29]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(descriptors)  
multiimp_descriptors=pd.DataFrame(imp.transform(descriptors))
multiimp_descriptors.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169
0,76698.0,7.800059,2.130706e+09,280.000000,0.0,0.0,0.0,0.0,0.0,0.0,37250.0,1432864.0,3664156.0,1007684.0,25896.0,0.0,2551696.0,0.0,0.0,0.0,0.0,0.0,4933296.0,3655166.0,1766008.0,1132040.0,0.0,0.0,0.0,0.0,1012.0,268.0,0.0,0.0,0.0,0.0,0.0,469014.0,4239660.0,703300.0,755876.0,0.0,5374.0,2108.0,4114.0,12348.0,615248.0,5526276.0,2378.0,4.0,0.0,0.0,2328746.0,1022304.0,415432.0,287230.0,310246.0,681504.0,1118814.0,3574.0,0.0,0.0,6700214.0,0.0,10.0,108.0,50.0,2551696.0,97518.0,947550.0,799478.0,330760.00000,353400.000000,299160.000000,305200.000000,283680.000000,270993.969114,361316.776657,304806.408322,178540.0,76698.078125,6700214.0,6700214.0,6599892.0,43566.0,68656.0,54064.0,638360.0,6167850.0,1209600.0,246244.0,2.000000,96.000000,0.000000,5.245752e+06,0.0,916567.687500,6.0,1924.0,0.0,0.0,0.0,118196.0,1309472.0,3247182.0,1381362.0,98822.0,11208.0,1608.0,220.000000,240.0,6700214.0,3294.902034,10476.0,1226.0,267998.0,521832.0,428776.0,4015854.0,895240.0,26330.0,118.0,0.0,532.00000,734.000000,4.122704e+06,51288.00000,0.000000,532572.000000,0.000000,18.000000,5.330690e+06,4732.0,1126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62282.0,85908.0,32790.0,0.0,0.0,202710.0,37928.0,14745580.0,1876644.0,0.0,0.0,0.0,0.0,2801180.0,2445.800049,2712.0,965866.0,1706908.0,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,33058.0,11.692895,0.000000e+00,191449.356064,0.0,0.0,0.0,0.0,0.0,0.0,18254.0,653294.0,1720800.0,516724.0,31642.0,0.0,1393352.0,0.0,68.0,0.0,0.0,0.0,2560898.0,2127150.0,1084598.0,338544.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71510.0,772720.0,1996924.0,99560.0,0.0,7336.0,7808.0,13776.0,13086.0,1010074.0,1873902.0,14726.0,6.0,0.0,0.0,1378576.0,447166.0,199512.0,154298.0,137280.0,138668.0,165908.0,229652.0,87082.0,4708.0,3646660.0,86.0,454.0,364.0,350.0,1393352.0,49028.0,688314.0,392208.0,341420.00000,359780.000000,366560.000000,495307.473852,526032.738215,611133.032134,569223.008629,554209.974697,6700.0,33057.511719,3646660.0,3646660.0,3582034.0,17733.0,260120.0,115626.0,6900.0,2942850.0,1209600.0,0.0,188073.487774,229.065148,0.023026,2.291079e+06,0.0,643536.937500,0.0,0.0,0.0,0.0,38.0,98644.0,1179502.0,1286736.0,336388.0,36294.0,5192.0,56.0,187415.448905,0.0,3646660.0,-133.956678,6160.0,796.0,164860.0,350066.0,272956.0,1837600.0,301242.0,9148.0,22.0,0.0,12112.67347,3460.545906,1.504309e+06,-140226.72134,6662.969238,54547.038888,-3.077087,51.269632,3.240763e+06,3312.0,522.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33736.0,36946.0,5936.0,0.0,0.0,103330.0,16254.0,4510080.0,868538.0,0.0,0.0,0.0,0.0,3477820.0,2211.760010,2334.0,664504.0,824154.0,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,41040.0,2.791966,2.280000e+02,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,1648.0,370592.0,1883374.0,292936.0,12016.0,0.0,1234132.0,0.0,0.0,0.0,0.0,0.0,2371990.0,2173634.0,300796.0,153698.0,0.0,0.0,0.0,0.0,358.0,110.0,0.0,0.0,0.0,0.0,0.0,0.0,870456.0,239798.0,1450312.0,0.0,1620.0,1156.0,1228.0,34250.0,1811606.0,710672.0,34.0,0.0,0.0,0.0,790690.0,672026.0,332340.0,254892.0,189596.0,135758.0,103552.0,81666.0,46.0,0.0,2673338.0,128.0,202.0,576.0,4.0,1234132.0,28804.0,160176.0,139730.0,137160.00000,130640.000000,210384.623016,307756.205172,376617.620151,445876.295962,493261.013255,522670.507914,28000.0,41040.078125,2673338.0,2673338.0,2678534.0,15439.0,7466.0,22436.0,248240.0,2560566.0,1209600.0,63328.0,0.000000,124.000000,0.000000,2.322692e+06,0.0,236099.515625

In [39]:
from sklearn.preprocessing import StandardScaler
scaled_mi_descriptors = StandardScaler().fit_transform(multiimp_descriptors)

In [40]:
pca = decomposition.PCA(n_components=5)
pca.fit(multiimp_descriptors)  
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

[9.99968101e-01 1.90864155e-05 6.79098537e-06 4.41760231e-06
 8.01257361e-07]
[6.76441283e+12 2.95528397e+10 1.76280157e+10 1.42177325e+10
 6.05512409e+09]


In [ ]:
count = 1
np.seterr(divide = 'ignore', invalid = 'ignore')
sns.set(font_scale=3)
plt.figure(figsize = (40,40))
plt.subplot(2,2,count)
vHeader = norm_df[name]
pca = PCA(n_components=len(PC))
pca.fit(vHeader)
comps = pd.DataFrame(pca.components_, columns = vHeader.columns)
# Delete columns with zero values
pos = 0
for colCount in range(0, len(comps.columns.values)-1):
    columnLabel = comps.columns[pos]
    data = comps.loc[:, comps.columns[pos]]
    if all([((v < .25) & (v > -.25)) for v in data]):
        comps.drop(columnLabel, axis = 1, inplace = True)
    else:
        pos = pos + 1
# Graph
heatmap = sns.heatmap(comps, center = 0, xticklabels = 1)
plt.title(name +' PCA')
count = count + 1 # Iteration
# Saving to PDF
fig = heatmap.get_figure()
fig.savefig(filePath + 'data/plots/PCA.pdf')

In [15]:
pca = decomposition.PCA()
data_pca = pca.fit_transform(descriptors)
evr = pca.explained_variance_ratio_
PC = [] # empty array to store EVR's in until 70
count = 0;
while sum(PC) < .70: # adds indexes from evr until over 70%
    PC.append(evr[count])
    count = count + 1

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [6]:
filter_col = [col for col in df_train if col.startswith('sensor')]
descriptors = df_train[filter_col].astype('float32')
target=df_train["target"].astype('float32')

TypeError: 'NoneType' object is not iterable